In [181]:
# from google.colab import drive
# drive.mount('/content/drive')

In [182]:
import os
print(os.getcwd())

/Users/swarajyabhanja/Desktop/MLOps_Assignments/group10_mlopsassignment1


In [183]:
import pandas as pd

In [184]:
dataset = pd.read_csv('/Users/swarajyabhanja/Desktop/MLOps_Assignments/insurance.csv')

In [185]:
df= pd.DataFrame(dataset)

In [186]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [187]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [188]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import Ridge, Lasso

In [189]:
import joblib
import warnings

In [190]:
warnings.filterwarnings('ignore')

In [191]:
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])
df['smoker'] = le.fit_transform(df['smoker'])
df['region'] = le.fit_transform(df['region'])

# Independent and dependent variables
x = df[["age", "bmi", "children", "sex", "smoker", "region"]]
y = df['charges']

In [192]:
# Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [193]:
# Train model
def train_model(n_estimators, max_depth):
    with mlflow.start_run():
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)

        # Log parameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Log metrics
        mse = mean_squared_error(y_test, predictions)
        mlflow.log_metric("mse", mse)

        # Log model
        mlflow.sklearn.log_model(model, "model")
        print(f"Model with n_estimators={n_estimators} and max_depth={max_depth} logged.")


In [194]:
# Run experiments 1
train_model(100, 3)

2024/08/07 14:30:02 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model with n_estimators=100 and max_depth=3 logged.


In [195]:
# Run experiments 2
train_model(200, 5)

2024/08/07 14:30:07 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model with n_estimators=200 and max_depth=5 logged.


In [196]:
# Run experiments 3
train_model(300, 7)


2024/08/07 14:30:12 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model with n_estimators=300 and max_depth=7 logged.


View experiment logs:
use the MLflow UI.

In [197]:
import subprocess

subprocess.run(["mlflow", "ui", "--port", "5000"])

[2024-08-07 14:30:15 +0530] [73772] [INFO] Starting gunicorn 22.0.0
[2024-08-07 14:30:15 +0530] [73772] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:30:15 +0530] [73772] [ERROR] Retrying in 1 second.
[2024-08-07 14:30:16 +0530] [73772] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:30:16 +0530] [73772] [ERROR] Retrying in 1 second.
[2024-08-07 14:30:17 +0530] [73772] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:30:17 +0530] [73772] [ERROR] Retrying in 1 second.
[2024-08-07 14:30:18 +0530] [73772] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:30:18 +0530] [73772] [ERROR] Retrying in 1 second.
[2024-08-07 14:30:19 +0530] [73772] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:30:19 +0530] [73772] [ERROR] Retrying in 1 second.
[2024-08-07 14:30:20 +0530] [73772] [ERROR] Can't connect to ('127.0.0.1', 5000)
Running the mlflow server failed. Please see the logs above for details.


CompletedProcess(args=['mlflow', 'ui', '--port', '5000'], returncode=1)

Data Versioning with DVC

In [198]:
pip install dvc

Note: you may need to restart the kernel to use updated packages.


Initialize a GIT repository:

In [199]:
import subprocess

subprocess.run(["git", "init"])

Reinitialized existing Git repository in /Users/swarajyabhanja/Desktop/MLOps_Assignments/group10_mlopsassignment1/.git/


CompletedProcess(args=['git', 'init'], returncode=0)

In [201]:
def run_command(command):
    try:
        result = subprocess.run(command, check=True, text=True, capture_output=True)
        print(f"Command '{' '.join(command)}' executed successfully.")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error executing command '{' '.join(command)}':")
        print(e.stderr)


In [202]:
# Step 1: Clone the repository
run_command(["git", "clone", "https://github.com/2022ac05599/group10_mlopsassignment1.git"])

# Change directory to the cloned repository
repo_name = "group10_mlopsassignment1"  # The name of the directory after cloning
os.chdir(repo_name)

# Step 2: Rename the current branch to 'main'
run_command(["git", "branch", "-M", "main"])

Command 'git clone https://github.com/2022ac05599/group10_mlopsassignment1.git' executed successfully.

Command 'git branch -M main' executed successfully.



Initialize a DVC repository

In [203]:
import subprocess

subprocess.run(["dvc", "init"])

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


CompletedProcess(args=['dvc', 'init'], returncode=0)

Add your dataset to DVC

In [204]:
import os

os.makedirs('data', exist_ok=True)

df.to_csv('data/dataset.csv', index=False)


Add the dataset to DVC:

In [205]:
subprocess.run(["dvc", "add", "data/dataset.csv"])


⠋ Checking graph



To track the changes with git, run:

	git add data/dataset.csv.dvc data/.gitignore

To enable auto staging, run:

	dvc config core.autostage true


CompletedProcess(args=['dvc', 'add', 'data/dataset.csv'], returncode=0)

This will create a .dvc file for your dataset, which should be tracked with Git.

In [206]:
# Add the .dvc file and .gitignore to Git
subprocess.run(["git", "add", "data/dataset.csv.dvc", ".gitignore"])

# Commit the changes
subprocess.run(["git", "commit", "-m", "Add dataset to DVC"])


[main (root-commit) 2254a3b] Add dataset to DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


fatal: pathspec '.gitignore' did not match any files


CompletedProcess(args=['git', 'commit', '-m', 'Add dataset to DVC'], returncode=0)

Make changes to the dataset:
Modify data/dataset.csv to create a new version.

In [207]:
# Step 3: Push the 'main' branch to the remote repository
run_command(["git", "push", "-u", "origin", "main"])

Command 'git push -u origin main' executed successfully.
Branch 'main' set up to track remote branch 'main' from 'origin'.



In [208]:
# Update the dataset
df.iloc[0, 2] = 37.000
df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,0,37.000,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


Modify data/dataset.csv to create a new version.

In [209]:
df.to_csv('data/dataset.csv', index=False)

Add the new version to DVC:

In [210]:
# Add the updated dataset to DVC
subprocess.run(["dvc", "add", "data/dataset.csv"])


⠋ Checking graph



To track the changes with git, run:

	git add data/dataset.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


CompletedProcess(args=['dvc', 'add', 'data/dataset.csv'], returncode=0)

In [211]:
# Add the .dvc file to Git
subprocess.run(["git", "add", "data/dataset.csv.dvc"])



CompletedProcess(args=['git', 'add', 'data/dataset.csv.dvc'], returncode=0)

In [212]:
# Commit the changes to Git
subprocess.run(["git", "commit", "-m", "Update dataset to new version"])

[main 01f3130] Update dataset to new version
 1 file changed, 5 insertions(+)
 create mode 100644 data/dataset.csv.dvc


CompletedProcess(args=['git', 'commit', '-m', 'Update dataset to new version'], returncode=0)

In [213]:
# Step 3: Push the 'main' branch to the remote repository
run_command(["git", "push", "-u", "origin", "main"])

Command 'git push -u origin main' executed successfully.
Branch 'main' set up to track remote branch 'main' from 'origin'.



Revert to a previous version:

In [214]:
# Checkout the previous version of the .dvc file
subprocess.run(["git", "checkout", "HEAD^1", "data/dataset.csv.dvc"])


error: pathspec 'data/dataset.csv.dvc' did not match any file(s) known to git


CompletedProcess(args=['git', 'checkout', 'HEAD^1', 'data/dataset.csv.dvc'], returncode=1)

In [215]:

# Use DVC to checkout the previous version of the dataset
subprocess.run(["dvc", "checkout"])

CompletedProcess(args=['dvc', 'checkout'], returncode=0)

In [216]:
# Run the git log command
result = subprocess.run(["git", "log", "--oneline"], capture_output=True, text=True)

# Print the output
print(result.stdout)

01f3130 Update dataset to new version
2254a3b Add dataset to DVC



MLflow Experiment Logs:

In [221]:
# Path where you want to store the mlruns directory
mlruns_path = 'file:///Users/swarajyabhanja/Desktop/MLOps_Assignments/mlruns'


In [223]:
# Start the MLflow UI
subprocess.run(["mlflow", "ui", "--backend-store-uri", mlruns_path])

[2024-08-07 14:41:12 +0530] [74029] [INFO] Starting gunicorn 22.0.0
[2024-08-07 14:41:12 +0530] [74029] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:41:12 +0530] [74029] [ERROR] Retrying in 1 second.
[2024-08-07 14:41:13 +0530] [74029] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:41:13 +0530] [74029] [ERROR] Retrying in 1 second.
[2024-08-07 14:41:14 +0530] [74029] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:41:14 +0530] [74029] [ERROR] Retrying in 1 second.
[2024-08-07 14:41:15 +0530] [74029] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:41:15 +0530] [74029] [ERROR] Retrying in 1 second.
[2024-08-07 14:41:16 +0530] [74029] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-08-07 14:41:16 +0530] [74029] [ERROR] Retrying in 1 second.
[2024-08-07 14:41:17 +0530] [74029] [ERROR] Can't connect to ('127.0.0.1', 5000)
Running the mlflow server failed. Please see the logs above for details.


CompletedProcess(args=['mlflow', 'ui', '--backend-store-uri', 'file:///Users/swarajyabhanja/Desktop/MLOps_Assignments/mlruns'], returncode=1)